In [36]:
# Data Cleaning

import pandas as pd
import numpy as np
data = pd.read_csv('HR-Employee-Attrition.csv')
# print(data_raw)

In [47]:
#1 Convert text data to numerical (sklearn’s preprocessing module, use LabelEncoder() like in U6S4)
    # adapted from documentation: https://saturncloud.io/blog/how-to-convert-text-columns-into-numbers-in-scikitlearn/
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
features_to_encode = ['Attrition', 'BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'Over18', 'OverTime']
for feature in features_to_encode:
    data[feature] = encoder.fit_transform(data[feature])


In [ ]:
#2 Feature Selection
    # Use Chi-squared or some other method to find columns that are irrelevant and just generate noise
    # Boruta makes it easy to use filtering, use BorutaPy ("now part of scikit-learn-contrib"): https://neptune.ai/blog/feature-selection-methods
        # for implementation: https://towardsdatascience.com/feature-selection-with-boruta-in-python-676e3877e596 

In [ ]:
#3 Normalize

In [ ]:
#4 Extra Credit?: graphing

In [13]:
# Model Training

In [14]:
#1 Split the data (stratify too)

In [15]:
#2 K Nearnest Neighbors

In [16]:
#3 Decision Tree

In [17]:
#4 Linear Regression

In [18]:
#5 Support Vector Machines

In [19]:
#6 Artificial Neural Networks